In [1]:
import sys
import os
sys.path.append('../')
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import project_dir

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")


In [2]:
D, DHat, kappa = symbols('D DHat kappa')

Cl = Gen.Closure(kappa, D, DHat)

kB = - kappa * (D - DHat) * (D - DHat) * log(D / DHat)
kB

-kappa*(D - DHat)**2*log(D/DHat)

In [3]:
dkBdd = kB.diff(D)
dkBdd

-kappa*(2*D - 2*DHat)*log(D/DHat) - kappa*(D - DHat)**2/D

In [4]:
ddkBddd = dkBdd.diff(D)
ddkBddd



-2*kappa*log(D/DHat) - 2*kappa*(2*D - 2*DHat)/D + kappa*(D - DHat)**2/D**2

In [5]:
s = f'''
// > Squared Version
// > D := d*d 
// > DHat := dHat*dHat

{Cl("KappaBarrier",kB)}
{Cl("dKappaBarrierdD",dkBdd)}
{Cl("ddKappaBarrierddD",ddkBddd)}
'''
print(s)

f = open( project_dir() / 'src/uipc/backends/cuda/contact_system/contact_models/sym/ipc_contact.inl', 'w')
f.write(s)
f.close()


// > Squared Version
// > D := d*d 
// > DHat := dHat*dHat

template <typename T>
__host__ __device__ void KappaBarrier(T& R, const T& kappa, const T& D, const T& DHat)
{
/*****************************************************************************************************************************
Function generated by SymEigen.py 
Author: MuGdxy
GitHub: https://github.com/MuGdxy/SymEigen
E-Mail: lxy819469559@gmail.com
******************************************************************************************************************************
LaTeX expression:
//tex:$$R = - \kappa \left(D - \hat{D}\right)^{2} \log{\left(\frac{D}{\hat{D}} \right)}$$

Symbol Name Mapping:
kappa:
    -> {}
    -> Matrix([[kappa]])
D:
    -> {}
    -> Matrix([[D]])
DHat:
    -> {}
    -> Matrix([[DHat]])
*****************************************************************************************************************************/
/* Sub Exprs */
/* Simplified Expr */
R = -kappa*std::pow((D - DHat), (2))*lo